<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
df.duplicated().sum() # проверим на дубликаты

153

In [6]:
# удалим дубликаты
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [7]:
df.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.001953,0.015456,-0.007315,0.011565
Возраст,0.001953,1.000000,-0.017386,-0.009064,0.654964
Зарплата,0.015456,-0.017386,1.000000,-0.031687,-0.013123
Члены семьи,-0.007315,-0.009064,-0.031687,1.000000,-0.039303
Страховые выплаты,0.011565,0.654964,-0.013123,-0.039303,1.000000


**Вывод:**

- Все данные в датасете заполнены (пропусков нет).
- Типы данных изменений не требуют.
- Выявлены и удалены дубликаты.
- Матрица корреляций не выявила значительных зависимостей между обучающими признаками.

## Умножение матриц

**Вопрос:** Изменится ли качество линейной регрессии, если признаки умножают на обратимую матрицу. ?

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится. 

**Обоснование:**

1. Для обоснования ответа получим предсказания для домноженных признаков. Пусть наши признаки будут как в исходном датасете размерностью 5000 на 4, а матрица для домножения 4 на 4. Умноженная матрица будет иметь вид X * P. Тогда фомула предсказаний примет следующий вид:

$$ a'=XPw'$$

2. Подставим в формулу предсказаний формулу обучения для матрицы X P:

$$ a'=XP((XP)^T XP)^{-1}(XP)^Ty$$

Используем некоторые свойства матриц:

$$(AB)^{-1}=B^{-1}A^{-1}$$

$$(AB)^T=A^T B^T$$

$$AA^{-1}=E$$

$$AE=EA=A$$

3. раскроем скобки с транспонированием матриц, получится следующее уравнение:

$$a'=XP(P^T X^T XP)^{-1}P^T X^Ty$$

Внутри скобок, от множителей которых берется обратная матрица остались следующие матрицы: $P^T$,размером 4х4, $X^T$ размером 4х5000, $X$ размером 5000x4, $P$ размером 4x4. Одно из дополнительных свойств матриц, о котором следует помнить слудующее $A(BC)=(AB)C$. Оно позволяет группировать матрицы внутри произведения различным образом. Т.к. обратная матрица может браться только от квадратной матрицы, то для раскрытия скобок данного произведения нам надо сгруппировать его множители так, чтобы они были квадратными матрицами.

4. Выделим 2 множителя произведения $P^T X^T X$ и $P$ и раскроем скобки:

$$a'=XPP^{-1}(P^T X^T X)^{-1}P^T X^Ty$$

5. Выделим внутри оставшихся скобок множители $P^T$ и $X^T X$ и раскроем скобки:

$$a'=XPP^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^Ty$$

6. Произведения $PP^{-1}$ и $(P^T)^{-1} P^T$ дают единичные матрицы, которые можно исключить из уровнения. С учетом этого формула принимает вид:

$$a'=X(X^T X)^{-1}X^T y$$

Мы знаем, что произведение $(X^T X)^{-1}X^T y$ равно $w$ , тогда:

$$a'=Xw$$

**Вывод:**
Таким образом, мы получили аналогичную формулу предсказаний, что и была в начале, значит домножение признаков на обратимую матрицу не влияет на предсказания модели.

## Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования примем домножение обучающих признаков X на случайную обратимую матрицу P размером 4х4. Создадим модель линейной регрессии до преобразования и посчитаем метрику R2. Далее умножим исходные признаки на обратимую матрицу и на основе полученных значений вновь посчитаем метрику R2. Метрики R2 по итогу должны быть равны.

**Обоснование**

Как было показано в предыдущем пункте, домножение признаков на какую-либо обратимую матрицу не приводит к изменению предсказаний, поэтому результаты R2 должны оказаться равны.

## Проверка алгоритма

Подготовим необходимые признаки: обучающие признаки исходные, целевой признак, матрицу для умножения, обучающие признаки после умножения. После подготовки признаков проведем машинное обучение с помощью алгоритма линейной регрессии для признаков до умножения и после. Сделаем в каждом случае предсказания и определим метрики r2.

In [8]:
# создадим признаки для исходного состояния
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']
display(features.head())
display(target.head())

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

In [9]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2 = r2_score(target, predictions)
print(f'Метрика R2 для признаков в исходном состоянии равняется {r2}')

Метрика R2 для признаков в исходном состоянии равняется 0.4302010044852067


Далее необходимо создать обратимую матрицу размером 4 на 4. При этом достаточно важным является не создать случайной необратимую матрицу, т.к. это нарушит наш алгоритм. **По одной из теорем обратных матриц:** для существования обратной матрицы $A^{-1}$ необходимо и достаточно, чтобы матрица $A$ была невырожденной, то есть, чтобы ее определитель не был равен нулю. Заложим проверку данного условия при создании обратимой матрицы.

In [10]:
# создание обратимой матрицы
det = 0 
P = 0
while det == 0:
    P = np.random.normal(size=(4, 4))
    det = np.linalg.det(P)
print(P)

[[-1.17998941  1.13536035 -0.85620075  1.36936611]
 [-0.37726558  0.03716676 -0.0780923   0.91704008]
 [-0.87908772  0.62376425  0.76680568  2.60994845]
 [-0.74703071  0.87865642 -0.4211972   0.31162903]]


In [11]:
# умножим исходные признаки на случайную обратимую матрицу
features_multi = features @ P
features_multi.head()

,0,1,2,3
0,-43620.145849,30942.244831,38029.082654,129492.722769
1,-33423.434629,23705.629963,29134.602480,99220.536581
2,-18471.782834,13100.127161,16100.654649,54835.511617
3,-36667.374586,26013.507189,31973.314614,108854.731474
4,-22955.932932,16282.423048,20010.585520,68146.701039


Проверим модель на измененных признаках.

In [12]:
model_2 = LinearRegression()
model_2.fit(features_multi, target)
predictions = model_2.predict(features_multi)
r2_multi = r2_score(target, predictions)
print(f'Метрика R2 для измененных признаков равняется {r2_multi}')

Метрика R2 для измененных признаков равняется 0.4302010044852378


**Вывод:** Метрики R2 обеих моделей одиннаковые, следовательно умножение исходных данных на случайную обратимую матрицу может быть применено для их шифрования.